In [1]:
import os
import sys
import math
import re
import time
import functools
import itertools
from abc import ABC
from dataclasses import dataclass, field
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
from scipy.linalg import solve
from scipy.stats import norm
from scipy.optimize import minimize
import matplotlib.pyplot as plt
!{sys.executable} -m pip install pyfinance -q
from pyfinance.options import BSM as BSMAux

Add any general utility functions as static member functions to this class. They can be called as `Util.fn`.

In [2]:
class Util:
    def __init__(self):
        raise TypeError("Non-instantiable class")
        
    def __new__(self, *args, **kwargs):
        raise TypeError("Non-instantiable class")
        
    @staticmethod
    def make_regex_group_disjunction(coll):
        return "|".join(map(lambda x: f"({str(x)})", coll))

In [3]:
class OptionsData:
    data_dir = "data"
    default_filename = "isx2010C.xls"
    
    def __init__(self, filename=default_filename, clean=True):
        filepath = os.path.join(self.data_dir, filename)
        if not os.path.isfile(filepath):
            faulty_filepath = filepath
            filepath = os.path.join(self.data_dir, self.default_filename)
            print(f"[{type(self).__name__}] Warning: could not find {faulty_filepath!r}; proceeding with {filepath!r}")
        self.__sheet_df_dict = pd.read_excel(filepath, sheet_name=None)
        sheets = list(self.__sheet_df_dict.keys())
        self.__sheet_succ = dict(zip(sheets, sheets[1:] + [sheets[-1]]))
        if clean:
            for key, val in self.__sheet_df_dict.items():
                self.__sheet_df_dict[key] = self.__clean_df(val)
                
    def __get_item__(self, key):
        return self.__sheet_df_dict[key]
    
    def get_sheet_names(self):
        return list(self.__sheet_succ.keys())
    
    def get_sheet_df_dict(self):
        return self.__sheet_df_dict
    
    def get_next_sheet_name(self, sheet_name):
        assert sheet_name in self.__sheet_succ
        return self.__sheet_succ[sheet_name]
    
    def get_df(self, E=None, sheet_name=""):
        if not sheet_name:
            sheet_name = list(self.__sheet_df_dict.keys())[0]
            print(f"[{type(self).__name__}] Warning: sheet name not specified; proceeding with {sheet_name!r}")
        df = self.__sheet_df_dict[sheet_name]
        common = ["T", "T_norm", "S", "r"]
        if not E:
            return df[[*common, *filter(lambda x: re.match(r"[0-9]+", x), df.columns)]]
        strikes = E if type(E) is list or type(E) is tuple else [E]
        cols = [*common, *map(lambda x: str(int(x)), strikes)]
        return df[cols]
    
    def __clean_df(self, df):
        # Discard rows where no options data is available.
        df = df.dropna(how="all")
        # Rename the columns according to the following convention:
        #  T = Time to Maturity
        #  S = Price of the Underlying
        #  r = Risk-Free Interest Rate
        df = df.rename(lambda x: self.__rename_df_cols(str(x), df), axis="columns")
        # Adjust the interest rate properly.
        df["r"] = df["r"] / 100
        # Add new column with annual-normalized T (252 = no. trading days in a year).
        df["T_norm"] = df["T"] / 252
        # Re-arrange the columns.
        common = ["S", "r", "T", "T_norm"]
        cols = [*common, *filter(lambda x: re.search("[0-9]+", x), df.columns.astype(str))]
        return df[cols]
    
    def __rename_df_cols(self, col_name, df):
        ncol = len(df.columns)
        # Time to maturity | (price of the underlying | risk-free rate).
        regex = r"(?P<T>[0-9]+(-[0-9]{2}){2} ([0-9]{2}:){2}[0-9]{2})|(?P<Sr>Unnamed: (?P<idx>[0-9]+))"
        match = re.match(regex, col_name)
        if not match:
            return col_name
        if match["T"]:
            return "T"
        elif match["Sr"]:
            col_idx = int(match["idx"])
            # Third last depicts the price of the underlying...
            if col_idx == ncol - 3:
                return "S"
            # ...and the second last the risk free rate.
            elif col_idx == ncol - 2:
                return "r"


In [4]:
data = OptionsData("isx2010C.xls")
data.get_sheet_names()

['isx15012010C',
 'isx19022010C',
 'isx17122010C',
 'isx19112010C',
 'isx15102010C',
 'isx17092010C',
 'isx20082010C',
 'isx16072010C',
 'isx18062010C',
 'isx21052010C',
 'isx16042010C',
 'isx19032010C']

A class encapsulating the Black-Scholes-Merton model and related computations, such as Greeks. Can create instances from `pd.Series` objects (as returned by pd.DataFrame.iterrows) via the `BSM.make_from_series` factory method.

In [5]:
@dataclass(frozen=True)
class BSM:
    S: float
    E: float
    r: float
    T: float
    C_obs: float
    sigma: float = 1.0
    d1: float = field(init=False)
    d2: float = field(init=False)
    
    def __post_init__(self):
        S, E, r, T, C_obs, sigma = self.S, self.E, self.r, self.T, self.C_obs, self.sigma
        sigma = BSMAux(S0=S, K=E, T=T, r=r, sigma=0.5, kind='call').implied_vol(C_obs)
        object.__setattr__(self, "sigma", sigma)
        eps = np.finfo(float).eps
        d1 = (math.log(S / E) + (r + 0.5 * self.sigma**2) * T) / (sigma * math.sqrt(T) + eps)
        object.__setattr__(self, "d1", d1)
        d2 = self.d1 - self.sigma * math.sqrt(T)
        object.__setattr__(self, "d2", d2)
        
    @staticmethod    
    def make_from_dict(d, E):
        return BSM(d["S"], int(E), d["r"], d["T_norm"], d[E])
    
    @staticmethod
    def make_from_series(ser, E, sigma=1.0):
        ser = ser.filter(regex=Util.make_regex_group_disjunction(["S", int(E), "r", "T_norm"]), axis="index")
        assert ser.shape[0] == 4, f"[{type(self).__name__}] Error: The Series should have an index of form [S, E, r, T_norm], got {ser.index}."
        S, r, T, C_obs = ser.array
        return BSM(S, E, r, T, C_obs, sigma=sigma)
    
    @functools.cached_property
    def delta(self):
        return norm.cdf(self.d1)
    
    @functools.cached_property
    def gamma(self):
        return norm.pdf(self.d1) / (self.S * self.sigma * math.sqrt(self.T))
    
    @functools.cached_property
    def theta(self):
        S, E, r, T, sigma, d1, d2 = self.S, self.E, self.r, self.T, self.sigma, self.d1, self.d2
        return -0.5 * S * norm.pdf(d1) * sigma / math.sqrt(T) - r * E * math.exp(-r * T) * norm.cdf(d2)
    
    @functools.cached_property
    def vega(self):
        return self.S * math.sqrt(self.T) * norm.pdf(self.d1)


In [6]:
"""test = data.get_df()
td = test.to_dict("index")
for t, row in td.items():
    print(f"Day {t}")
    strikes = list(filter(lambda x: re.match(r"[0-9]+", x), row.keys()))
    BSMs = {}
    for E in strikes:
        BSMs[E] = BSM.make_from_dict(row, E)
    greeks = pd.DataFrame(index=strikes)
    greeks["delta"] = pd.Series([BSMs[E].delta for E in greeks.index], index=greeks.index)
    greeks["gamma"] = pd.Series([BSMs[E].gamma for E in greeks.index], index=greeks.index)
    greeks["vega"] = pd.Series([BSMs[E].vega for E in greeks.index], index=greeks.index)
    print(greeks[greeks > 0.001])
    print("-" * 50)
  """


'test = data.get_df()\ntd = test.to_dict("index")\nfor t, row in td.items():\n    print(f"Day {t}")\n    strikes = list(filter(lambda x: re.match(r"[0-9]+", x), row.keys()))\n    BSMs = {}\n    for E in strikes:\n        BSMs[E] = BSM.make_from_dict(row, E)\n    greeks = pd.DataFrame(index=strikes)\n    greeks["delta"] = pd.Series([BSMs[E].delta for E in greeks.index], index=greeks.index)\n    greeks["gamma"] = pd.Series([BSMs[E].gamma for E in greeks.index], index=greeks.index)\n    greeks["vega"] = pd.Series([BSMs[E].vega for E in greeks.index], index=greeks.index)\n    print(greeks[greeks > 0.001])\n    print("-" * 50)\n  '

In [ ]:
t = 0
strike_step = 5
S = td[t]["S"]
S
strikes = list(filter(lambda x: re.match(r"\d+", x), td[t].keys()))
strike_to_buy = strikes[np.argmin(np.abs(np.array(strikes, dtype=np.int64) - S))]
strike_to_buy
Gs = greeks[greeks > 0.001].dropna(how='all')
strike_to_buy
Gs = Gs.fillna(0)
print(Gs)
# G*w = p
p = Gs.loc[strike_to_buy].to_numpy()[0]
G = p
res = solve(G, p)
p - res*p

In [24]:

eps = np.finfo(float).eps

class Hedger:
    @dataclass
    class HedgingStats:
        cost_basis: float
        mse: float = 0.0
        total_cost: float = 0.0
            
        def __repr__(self):
            return (f"[Hedger.{type(self).__name__}]: "
                    f"Assuming a cost basis of {self.cost_basis*100:.2f}%, mean-squared error "
                    f"of hedging was {self.mse:.2f}, and the total costs were ${self.total_cost:.2f}.")
    
    @dataclass
    class DeltaState:
        long: float
        short: float
        delta: float
        E_to_delta: dict
            
    @dataclass
    class DeltaVegaState:
        portfolio: float
        underlying: float
        rep_option: float
        alpha: float
        eta: float

    def delta_hedge(self, data, sheet_name="", portfolio_size=2, schedule=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        strikes_considered = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size]]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, strikes_considered.index))}"))
        
        # Compute and save state for the required computations.
        BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in strikes_considered.index}
        deltas = np.sum(np.nan_to_num(list(map(lambda x: x.delta, BSMs.values()))))
        longs = day0[strikes_considered.index].sum()
        state_prev = Hedger.DeltaState(
            longs, deltas * day0.S, deltas, {E: BSMs[E].delta for E in BSMs.keys()}
        )
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.short
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            BSMs = {E: BSM.make_from_dict(row, E) for E in strikes_considered.index}
            longs = sum(map(lambda x: x.C_obs, BSMs.values()))
            delta_handler = lambda x: x.delta if not math.isnan(x.delta) else state_prev.E_to_delta[str(x.E)]
            state = Hedger.DeltaState(
                longs, state_prev.delta * row["S"], state_prev.delta, {E: delta_handler(BSMs[E]) for E in BSMs.keys()}
            )
            dlong = state.long - state_prev.long
            dshort = state.short - state_prev.short
            squared_errors.append((dlong - dshort)**2)
            # Rehedge?
            if t % schedule == 0:
                deltas = sum(state.E_to_delta.values())
                state.short = deltas * row["S"]
                state.delta = deltas
                stats.total_cost += abs(cost_basis * (state_prev.delta - state.delta) * row["S"])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats
        
    def delta_vega_hedge(self, data, schedule=2, sheet_name="", sheet_name2="isx19022010C", portfolio_size=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        df2 = data.get_df(sheet_name=sheet_name2)
        n_options_considered = 2
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        strikes_considered = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size+1]]
        rep_option_strike = strikes_considered[-1:]
        portfolio_strikes = strikes_considered[:-1]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, portfolio_strikes.index))}"))
        
        # Compute and save state for the required computations.
        portfolio_BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in portfolio_strikes.index}
        rep_E = rep_option_strike.index[0]
        rep_option_BSM = BSM(day0.S, int(rep_E), day0.r, day0.T_norm, day0[rep_E])
        portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
        portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
        rep_option_delta = rep_option_BSM.delta
        rep_option_vega = rep_option_BSM.vega
        portfolio = day0[portfolio_strikes.index].sum() 
        alpha = -portfolio_delta + portfolio_vega / rep_option_vega * rep_option_delta
        eta = -portfolio_vega / rep_option_vega
        # don't hedge at day 0 if some delta or vega is nan (=implied vol is nan)
        if math.isnan(alpha+eta):
            alpha, eta = 0, 0
        state_prev = Hedger.DeltaVegaState(portfolio, alpha * day0.S, eta * day0[rep_E], alpha, eta)
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.alpha * day0.S + cost_basis * state_prev.eta * day0[rep_E]
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            portfolio_BSMs = {E: BSM.make_from_dict(row, E) for E in portfolio_strikes.index}
            rep_option_BSM = BSM.make_from_dict(row, rep_E)
            portfolio = sum(map(lambda x: x.C_obs, portfolio_BSMs.values()))
            state = Hedger.DeltaVegaState(portfolio, state_prev.alpha * row["S"], state_prev.eta * row[rep_E], state_prev.alpha, state_prev.eta)
            diff = (state.portfolio + state.underlying + state.rep_option) - (state_prev.portfolio + state_prev.underlying + state_prev.rep_option)
            squared_errors.append((diff)**2)
            # Rehedge?
            if t % schedule == 0:
                portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
                portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
                rep_option_delta = rep_option_BSM.delta
                rep_option_vega = rep_option_BSM.vega
                alpha = -portfolio_delta + portfolio_vega / (rep_option_vega + eps) * rep_option_delta
                eta = -portfolio_vega / (rep_option_vega + eps)
                # if alpha or eta is null use previous
                if not math.isnan(alpha+eta):
                    state.alpha = alpha
                    state.eta = eta
                state.underlying = state.alpha * row["S"]
                state.rep_option = state.eta * row[rep_E]
                stats.total_cost += abs(cost_basis * (state_prev.alpha - state.alpha) * row["S"] + cost_basis * (state_prev.eta - state.eta) * row[rep_E])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats

In [25]:
h = Hedger()
h.delta_hedge(data, portfolio_size=2)

[OptionsData] Warning: sheet name not specified; proceeding with 'isx15012010C'
[Hedger] Info: Considering a position in call(s) with strike price(s) of $500, $480


[Hedger.HedgingStats]: Assuming a cost basis of 1.00%, mean-squared error of hedging was 3.55, and the total costs were $24.26.

In [149]:
# Doing replicated option from another sheet here



eps = np.finfo(float).eps

class Hedger:
    @dataclass
    class HedgingStats:
        cost_basis: float
        mse: float = 0.0
        total_cost: float = 0.0
            
        def __repr__(self):
            return (f"[Hedger.{type(self).__name__}]: "
                    f"Assuming a cost basis of {self.cost_basis*100:.2f}%, mean-squared error "
                    f"of hedging was {self.mse:.2f}, and the total costs were ${self.total_cost:.2f}.")
    
    @dataclass
    class DeltaState:
        long: float
        short: float
        delta: float
            
    @dataclass
    class DeltaVegaState:
        portfolio: float
        underlying: float
        rep_option: float
        alpha: float
        eta: float

    def delta_hedge(self, data, sheet_name="", portfolio_size=2, schedule=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        strikes_considered = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size]]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, strikes_considered.index))}"))
        
        # Compute and save state for the required computations.
        BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in strikes_considered.index}
        deltas = np.sum(np.nan_to_num(list(map(lambda x: x.delta, BSMs.values()))))
        longs = day0[strikes_considered.index].sum()
        state_prev = Hedger.DeltaState(longs, deltas * day0.S, deltas)
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.short
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            BSMs = {E: BSM.make_from_dict(row, E) for E in strikes_considered.index}
            longs = sum(map(lambda x: x.C_obs, BSMs.values()))
            state = Hedger.DeltaState(longs, state_prev.delta * row["S"], state_prev.delta)
            dlong = state.long - state_prev.long
            dshort = state.short - state_prev.short
            squared_errors.append((dlong - dshort)**2)
            # Rehedge?
            if t % schedule == 0:
                deltas = np.sum(np.nan_to_num(list(map(lambda x: x.delta, BSMs.values()))))
                state.short = deltas * row["S"]
                state.delta = deltas
                stats.total_cost += abs(cost_basis * (state_prev.delta - state.delta) * row["S"])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats
        
    def delta_vega_hedge(self, data, schedule=2, sheet_name="", sheet_name2="isx19022010C", portfolio_size=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        df2 = data.get_df(sheet_name=sheet_name2)
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        day0_2 = df2.iloc[0]
        strikes2 = day0_2.dropna().filter(regex=r"\d+").index
        option_value_ser_2 = pd.Series(data=strikes2, index=strikes2, dtype=int).apply(lambda E: abs(day0_2.S - E))
        
        portfolio_strikes = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size]]
        rep_option_strike = option_value_ser_2.iloc[np.argsort(option_value_ser_2)[:1]]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, portfolio_strikes.index))}"))
        
        # Compute and save state for the required computations.
        portfolio_BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in portfolio_strikes.index}
        rep_E = rep_option_strike.index[0]
        rep_option_BSM = BSM(day0_2.S, int(rep_E), day0_2.r, day0_2.T_norm, day0_2[rep_E])
        portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
        portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
        rep_option_delta = rep_option_BSM.delta
        rep_option_vega = rep_option_BSM.vega
        portfolio = day0[portfolio_strikes.index].sum() 
        alpha = -portfolio_delta + portfolio_vega / rep_option_vega * rep_option_delta
        eta = -portfolio_vega / rep_option_vega
        # don't hedge at day 0 if some delta or vega is nan (=implied vol is nan)
        if math.isnan(alpha+eta):
            alpha, eta = 0, 0
        state_prev = Hedger.DeltaVegaState(portfolio, alpha * day0.S, eta * day0_2[rep_E], alpha, eta)
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.alpha * day0.S + cost_basis * state_prev.eta * day0_2[rep_E]
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        day1_onwards_2 = df2.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            row2 = day1_onwards_2[t]
            portfolio_BSMs = {E: BSM.make_from_dict(row, E) for E in portfolio_strikes.index}
            rep_option_BSM = BSM.make_from_dict(row2, rep_E)
            portfolio = sum(map(lambda x: x.C_obs, portfolio_BSMs.values()))
            state = Hedger.DeltaVegaState(portfolio, state_prev.alpha * row["S"], state_prev.eta * row2[rep_E], state_prev.alpha, state_prev.eta)
            diff = (state.portfolio + state.underlying + state.rep_option) - (state_prev.portfolio + state_prev.underlying + state_prev.rep_option)
            squared_errors.append((diff)**2)
            # Rehedge?
            if t % schedule == 0:
                portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
                portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
                rep_option_delta = rep_option_BSM.delta
                rep_option_vega = rep_option_BSM.vega
                alpha = -portfolio_delta + portfolio_vega / (rep_option_vega + eps) * rep_option_delta
                eta = -portfolio_vega / (rep_option_vega + eps)
                if not math.isnan(alpha+eta):
                    state.alpha = alpha
                    state.eta = eta
                state.underlying = state.alpha * row["S"]
                state.rep_option = state.eta * row2[rep_E]
                stats.total_cost += abs(cost_basis * (state_prev.alpha - state.alpha) * row["S"] + cost_basis * (state_prev.eta - state.eta) * row2[rep_E])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats


In [ ]:
# Doing replicated option from another sheet here



eps = np.finfo(float).eps

class Hedger:
    @dataclass
    class HedgingStats:
        cost_basis: float
        mse: float = 0.0
        total_cost: float = 0.0
            
        def __repr__(self):
            return (f"[Hedger.{type(self).__name__}]: "
                    f"Assuming a cost basis of {self.cost_basis*100:.2f}%, mean-squared error "
                    f"of hedging was {self.mse:.2f}, and the total costs were ${self.total_cost:.2f}.")
    
    @dataclass
    class DeltaState:
        long: float
        short: float
        delta: float
            
    @dataclass
    class DeltaVegaState:
        portfolio: float
        underlying: float
        rep_option: float
        alpha: float
        eta: float

    def delta_hedge(self, data, sheet_name="", portfolio_size=2, schedule=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        strikes_considered = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size]]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, strikes_considered.index))}"))
        
        # Compute and save state for the required computations.
        BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in strikes_considered.index}
        deltas = np.sum(np.nan_to_num(list(map(lambda x: x.delta, BSMs.values()))))
        longs = day0[strikes_considered.index].sum()
        state_prev = Hedger.DeltaState(longs, deltas * day0.S, deltas)
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.short
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            BSMs = {E: BSM.make_from_dict(row, E) for E in strikes_considered.index}
            longs = sum(map(lambda x: x.C_obs, BSMs.values()))
            state = Hedger.DeltaState(longs, state_prev.delta * row["S"], state_prev.delta)
            dlong = state.long - state_prev.long
            dshort = state.short - state_prev.short
            squared_errors.append((dlong - dshort)**2)
            # Rehedge?
            if t % schedule == 0:
                deltas = np.sum(np.nan_to_num(list(map(lambda x: x.delta, BSMs.values()))))
                state.short = deltas * row["S"]
                state.delta = deltas
                stats.total_cost += abs(cost_basis * (state_prev.delta - state.delta) * row["S"])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats
        
    def delta_vega_hedge(self, data, schedule=2, sheet_name="", sheet_name2="isx19022010C", portfolio_size=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        df2 = data.get_df(sheet_name=sheet_name2)
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        day0_2 = df2.iloc[0]
        strikes2 = day0_2.dropna().filter(regex=r"\d+").index
        option_value_ser_2 = pd.Series(data=strikes2, index=strikes2, dtype=int).apply(lambda E: abs(day0_2.S - E))
        
        portfolio_strikes = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size]]
        rep_option_strike = option_value_ser_2.iloc[np.argsort(option_value_ser_2)[:1]]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, portfolio_strikes.index))}"))
        
        # Compute and save state for the required computations.
        portfolio_BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in portfolio_strikes.index}
        rep_E = rep_option_strike.index[0]
        rep_option_BSM = BSM(day0_2.S, int(rep_E), day0_2.r, day0_2.T_norm, day0_2[rep_E])
        portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
        portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
        rep_option_delta = rep_option_BSM.delta
        rep_option_vega = rep_option_BSM.vega
        portfolio = day0[portfolio_strikes.index].sum() 
        alpha = -portfolio_delta + portfolio_vega / rep_option_vega * rep_option_delta
        eta = -portfolio_vega / rep_option_vega
        # don't hedge at day 0 if some delta or vega is nan (=implied vol is nan)
        if math.isnan(alpha+eta):
            alpha, eta = 0, 0
        state_prev = Hedger.DeltaVegaState(portfolio, alpha * day0.S, eta * day0_2[rep_E], alpha, eta)
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.alpha * day0.S + cost_basis * state_prev.eta * day0_2[rep_E]
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        day1_onwards_2 = df2.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            row2 = day1_onwards_2[t]
            portfolio_BSMs = {E: BSM.make_from_dict(row, E) for E in portfolio_strikes.index}
            rep_option_BSM = BSM.make_from_dict(row2, rep_E)
            portfolio = sum(map(lambda x: x.C_obs, portfolio_BSMs.values()))
            state = Hedger.DeltaVegaState(portfolio, state_prev.alpha * row["S"], state_prev.eta * row2[rep_E], state_prev.alpha, state_prev.eta)
            diff = (state.portfolio + state.underlying + state.rep_option) - (state_prev.portfolio + state_prev.underlying + state_prev.rep_option)
            squared_errors.append((diff)**2)
            # Rehedge?
            if t % schedule == 0:
                portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
                portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
                rep_option_delta = rep_option_BSM.delta
                rep_option_vega = rep_option_BSM.vega
                alpha = -portfolio_delta + portfolio_vega / (rep_option_vega + eps) * rep_option_delta
                eta = -portfolio_vega / (rep_option_vega + eps)
                if not math.isnan(alpha+eta):
                    state.alpha = alpha
                    state.eta = eta
                state.underlying = state.alpha * row["S"]
                state.rep_option = state.eta * row2[rep_E]
                stats.total_cost += abs(cost_basis * (state_prev.alpha - state.alpha) * row["S"] + cost_basis * (state_prev.eta - state.eta) * row2[rep_E])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats


In [150]:
not math.isnan(nan)

False

In [151]:
a, x = 0, 0

In [152]:
h = Hedger()
h.delta_vega_hedge(data, sheet_name='isx15012010C', sheet_name2='isx19022010C', portfolio_size=2)
#h.delta_hedge(data, portfolio_size=2)

[Hedger] Info: Considering a position in call(s) with strike price(s) of $500, $480


[Hedger.HedgingStats]: Assuming a cost basis of 1.00%, mean-squared error of hedging was 41.02, and the total costs were $39.83.

[OptionsData] Warning: sheet name not specified; proceeding with 'isx15012010C'
[Hedger] Info: Considering a position in call(s) with strike price(s) of $500


IndexError: invalid index to scalar variable.

In [7]:
df = data.get_df(sheet_name="")
        
# We consider at-the-money options.
day0 = df.iloc[0]
strikes = day0.dropna().filter(regex=r"\d+").index
option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
strikes_considered = option_value_ser.iloc[np.argsort(option_value_ser)[:5]]
rep_option_strike = strikes_considered[-1:]
portfolio_strikes = strikes_considered[:-1]

[OptionsData] Warning: sheet name not specified; proceeding with 'isx15012010C'


In [8]:
df

,T,T_norm,S,r,340,345,350,355,360,365,370,375,380,385,390,395,400,405,410,415,420,425,430,435,440,445,450,455,460,465,470,475,480,485,490,495,500,505,510,515,520,525,530,535,540,545,550,555,560,565,570
0,86,0.341270,491.34,0.0011,152.20,NaN,NaN,NaN,132.60,NaN,NaN,NaN,113.20,NaN,NaN,NaN,94.55,NaN,NaN,NaN,76.45,NaN,NaN,NaN,59.25,NaN,NaN,NaN,43.60,NaN,NaN,NaN,29.80,NaN,NaN,NaN,18.65,NaN,NaN,NaN,10.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85,0.337302,494.35,0.0011,155.05,NaN,NaN,NaN,135.35,NaN,NaN,NaN,115.90,NaN,NaN,NaN,96.90,NaN,NaN,NaN,78.75,NaN,NaN,NaN,61.45,NaN,NaN,NaN,45.45,NaN,NaN,NaN,31.40,NaN,NaN,NaN,19.80,NaN,NaN,NaN,11.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84,0.333333,490.36,0.0011,152.40,NaN,NaN,NaN,132.70,NaN,NaN,NaN,113.30,NaN,NaN,NaN,94.35,NaN,NaN,NaN,76.15,NaN,NaN,NaN,59.05,NaN,NaN,NaN,43.40,NaN,NaN,NaN,29.55,NaN,NaN,NaN,18.45,NaN,NaN,NaN,10.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,83,0.329365,486.99,0.0011,147.00,NaN,NaN,NaN,127.25,NaN,NaN,NaN,108.05,NaN,NaN,NaN,89.70,NaN,NaN,NaN,71.95,NaN,NaN,NaN,55.25,NaN,NaN,NaN,39.70,NaN,NaN,NaN,26.95,NaN,NaN,NaN,16.45,NaN,NaN,NaN,8.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,82,0.325397,484.11,0.0011,145.55,NaN,NaN,NaN,125.95,NaN,NaN,NaN,106.75,NaN,NaN,NaN,88.10,NaN,NaN,NaN,70.35,NaN,NaN,NaN,53.65,NaN,NaN,NaN,38.60,NaN,NaN,NaN,25.70,NaN,NaN,NaN,15.70,NaN,NaN,NaN,7.95,NaN,NaN,NaN,3.80,NaN,NaN,NaN,1525.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,5,0.019841,524.29,0.0006,185.50,180.50,175.50,170.50,165.50,160.50,155.50,150.50,145.50,140.50,135.50,130.50,125.50,120.50,115.50,110.50,105.50,100.50,95.50,90.50,85.40,80.40,75.40,70.40,65.40,60.40,55.40,50.45,45.45,40.45,35.45,30.50,25.55,20.00,15.70,11.0,6.80,3.00,1.00,0.30,0.10,0.07,0.05,0.05,0.05,0.05,0.05
82,4,0.015873,527.93,0.0006,188.55,183.55,178.55,173.55,168.55,163.55,158.55,153.55,148.55,143.55,138.55,133.55,128.55,123.55,118.55,113.55,108.55,103.55,98.55,93.55,88.55,83.55,78.55,73.55,68.70,63.55,58.55,53.55,48.60,43.65,38.65,33.65,28.60,23.65,18.65,14.0,9.00,4.50,1.60,0.40,0.15,0.07,0.05,0.05,0.05,0.15,0.15
83,3,0.011905,529.59,0.0006,190.10,185.10,180.10,175.10,170.10,165.10,160.10,155.10,150.10,145.10,140.10,135.10,130.10,125.10,120.10,115.10,110.10,105.10,100.10,95.10,90.10,85.10,80.10,75.10,70.10,65.10,60.10,55.10,50.30,45.10,40.10,35.10,30.80,26.00,20.00,15.7,10.10,5.65,1.95,0.40,0.05,0.05,0.02,0.05,0.05,0.15,0.05
84,2,0.007937,524.11,0.0005,184.10,179.10,174.10,169.10,164.10,159.10,154.10,149.10,144.10,139.10,134.10,129.10,124.10,119.10,114.10,109.10,104.10,99.10,94.10,89.10,84.10,79.10,74.10,69.10,63.10,59.10,54.10,49.10,45.50,41.25,34.58,27.80,24.00,19.40,14.10,9.1,4.10,0.05,0.05,0.04,0.05,0.05,0.01,0.05,0.05,0.05,0.05


In [58]:
day1_onwards = df.iloc[1:-1].to_dict("index")
for t, row in day1_onwards.items():
    break

In [ ]:
day1_onwards[1]

In [61]:
rep_E = rep_option_strike.index[0]
row[rep_E]

61.45

In [68]:
day0[rep_E]


59.25

In [55]:
day0[rep_option_strike.index][0]

59.25

In [49]:
8.66+11.34+28.66+31.34

80.0

In [50]:
portfolio_strikes

500     8.66
480    11.34
520    28.66
460    31.34
dtype: float64

In [63]:
E = 500
schedule = 2
df = data.get_df(E).dropna()

import time

t0 = time.time()
# t = 0:
rows_iterator = df.iterrows()
_, row = next(rows_iterator)
bsm_prev = BSM.make_from_series(row, E)
long_prev = bsm_prev.C_obs
delta_factor = bsm_prev.delta
short_prev = delta_factor * bsm_prev.S

# 0 < t < T:
mse = 0.0
for t, row in rows_iterator:
    bsm = BSM.make_from_series(row, E)
    long = bsm.C_obs
    dlong = long - long_prev
    short = delta_factor * bsm.S
    dshort = short - short_prev
    mse += (dlong - dshort)**2
    long_prev = long
    bsm_prev = bsm
    # Rehedge?
    if t % schedule == 0:
        delta_factor = bsm.delta
        short_prev = delta_factor * bsm.S
    else:
        short_prev = short

mse /= df.shape[0]

t1 = time.time()
print(f"Took {(t1 - t0)*1000:.2f} ms")

[OptionsData] Warning: sheet name not specified; proceeding with 'isx15012010C'


/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (


Took 4059.20 ms


In [64]:
print(f"Single option delta hedging {mse=:.2f}")

Single option delta hedging mse=nan
